## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-11-15-51-00 +0000,wsj,Major Trump Briefing on Iran Options Planned f...,https://www.wsj.com/world/middle-east/major-tr...
1,2026-01-11-15-49-46 +0000,nyt,Iran’s President Warns Protesters Not to Desta...,https://www.nytimes.com/2026/01/11/world/middl...
2,2026-01-11-15-35-21 +0000,bbc,"Trump won't take Greenland by force, Mandelson...",https://www.bbc.com/news/articles/cp9j1d3zgvgo...
3,2026-01-11-15-31-40 +0000,nypost,"Ex-New Jersey Gov. Richard Codey, who took ove...",https://nypost.com/2026/01/11/us-news/former-n...
4,2026-01-11-15-28-03 +0000,startribune,"In Minnesota’s most conservative county, GOP g...",https://www.startribune.com/minnesota-republic...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2345/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
1,trump,32
35,ice,19
3,iran,18
337,venezuela,14
157,protests,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
165,2026-01-10-17-56-09 +0000,nypost,Trump reposts Lindsey Graham’s tweet warning I...,https://nypost.com/2026/01/10/world-news/trump...,69
82,2026-01-11-03-57-21 +0000,nyt,Trump Is Briefed on Options for Striking Iran ...,https://www.nytimes.com/2026/01/10/us/politics...,68
75,2026-01-11-07-27-11 +0000,nypost,Iran warns US troops and Israel will be target...,https://nypost.com/2026/01/11/world-news/iran-...,67
132,2026-01-10-21-44-31 +0000,nypost,"New Jersey’s Petty’s Island, now owned by Vene...",https://nypost.com/2026/01/10/us-news/new-jers...,67
156,2026-01-10-19-06-36 +0000,nypost,Trump admin weighing preliminary attack plans ...,https://nypost.com/2026/01/10/world-news/trump...,67


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
165,69,2026-01-10-17-56-09 +0000,nypost,Trump reposts Lindsey Graham’s tweet warning I...,https://nypost.com/2026/01/10/world-news/trump...
80,57,2026-01-11-04-16-32 +0000,nyt,Anti-ICE Protests Spread Nationwide After Minn...,https://www.nytimes.com/2026/01/10/us/ice-shoo...
132,35,2026-01-10-21-44-31 +0000,nypost,"New Jersey’s Petty’s Island, now owned by Vene...",https://nypost.com/2026/01/10/us-news/new-jers...
75,35,2026-01-11-07-27-11 +0000,nypost,Iran warns US troops and Israel will be target...,https://nypost.com/2026/01/11/world-news/iran-...
142,32,2026-01-10-20-53-00 +0000,wsj,The Defense Department has selected Owen West ...,https://www.wsj.com/politics/national-security...
175,27,2026-01-10-17-00-00 +0000,wsj,In an industry town that has grown increasingl...,https://www.wsj.com/politics/policy/save-calif...
171,27,2026-01-10-17-23-27 +0000,nypost,US used powerful mystery weapon that brought V...,https://nypost.com/2026/01/10/world-news/us-us...
65,26,2026-01-11-10-00-00 +0000,wsj,President Trump’s proposals for the housing ma...,https://www.wsj.com/economy/housing/trumps-new...
87,25,2026-01-11-03-13-45 +0000,missionlocal,‘We’re here to create something positive’: Pop...,https://missionlocal.org/2026/01/were-here-to-...
28,24,2026-01-11-13-00-00 +0000,nypost,After Marine vet Trevor Reed was freed from a ...,https://nypost.com/2026/01/11/world-news/marin...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
